# ETL Notebook

This notebook is to do some DataFrame visualisation for our ETL process. We will run connect to the Database and run our SQL queries to extract our defined dimensions.

This can also be our testbed for potential transforms that we will be conducting for our NLP models.

In [45]:
# Import libraries
import mysql.connector
import pandas as pd
import os
from sqlalchemy import create_engine
from dotenv import load_dotenv
from uuid import uuid4
import pandas as pd
import numpy as np
from datetime import datetime
import re
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('vader_lexicon')
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer


import warnings
warnings.filterwarnings("ignore")

load_dotenv()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/joannelim/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/joannelim/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/joannelim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/joannelim/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [46]:
db_host = os.getenv("DATABASE_ENDPOINT")
db_user = os.getenv("DATABASE_USERNAME")
db_pw = os.getenv("DATABASE_PASSWORD")
db_name = os.getenv("DATABASE_NAME")
db_port = os.getenv("DATABASE_PORT")

dwh_host = os.getenv("DATAWH_ENDPOINT")
dwh_user = os.getenv("DATAWH_USERNAME")
dwh_pw = os.getenv("DATAWH_PASSWORD")
dwh_name = os.getenv("DATAWH_NAME")
dwh_port = os.getenv("DATAWH_PORT")

# housekeeping
db_datawarehouse = mysql.connector.connect(
	host=dwh_host,
	user=dwh_user,
	passwd=dwh_pw,
	database=dwh_name,
    auth_plugin=dwh_pw
)

cursor = db_datawarehouse.cursor()
cursor.execute('DROP TABLE IF EXISTS fact;')

cursor.execute('DROP TABLE IF EXISTS review;')
cursor.execute('DROP TABLE IF EXISTS time;')

engine = create_engine(f'mysql://{dwh_user}:{dwh_pw}@{dwh_host}:{dwh_port}/{dwh_name}', echo=False, future=True)
db_datawarehouse = engine.connect()

db_datawarehouse

In [47]:
db_tripadvisor = mysql.connector.connect(
	host=db_host,
	user=db_user,
	passwd=db_pw,
	database=db_name
)

engine = create_engine(f'mysql://{dwh_user}:{dwh_pw}@{dwh_host}:{dwh_port}/{dwh_name}', echo=False)

dwh = engine.connect()

# housekeeping
db_datawarehouse = mysql.connector.connect(
	host=dwh_host,
	user=dwh_user,
	passwd=dwh_pw,
	database=dwh_name,
    auth_plugin=dwh_pw
)

In [48]:
# Check if the OverallID column exists
cursor = db_tripadvisor.cursor()
cursor.execute("SHOW COLUMNS FROM tripadvisor_reviews LIKE 'OverallID'")
if cursor.fetchone() is None:
    # If OverallID column doesn't exist, add it
    cursor.execute('ALTER TABLE tripadvisor_reviews ADD OverallID INT AUTO_INCREMENT PRIMARY KEY')


In [49]:
# Check tripadvisor_reviews table

str_sql = '''
SELECT *
FROM tripadvisor_reviews
'''

df = pd.read_sql(sql=str_sql, con=db_tripadvisor)

df

,ReviewTitle,ReviewText,DateOfStay,AuthorContribution,Rating,OverallID
0,Must see in Singapore,A must not miss place for tourists to visit wh...,2024-03-01,73,5.0,1
1,I recently had the most rejuvenating spa,I recently had the most rejuvenating spa exper...,2024-03-01,3,5.0,2
2,Professional service,Visited the hotel for some drinks and what I r...,2024-03-01,2,5.0,3
3,Marina Bay world class,Amazing hotel and loved the facilities. Being ...,2024-03-01,1,5.0,4
4,Nice touch.,While the initial check in experience was not ...,2024-03-01,2,5.0,5
...,...,...,...,...,...,...
11227,A good hotel,Nice location and very near to a shopping mall...,2014-08-01,97,5.0,11228
11228,Swim in heaven,Everytime I check into the Sand I love going u...,2015-03-01,13,4.0,11229
11229,Unmatcheable Property,I stayed here for 2 nights in November'14. ...,2015-01-01,68,5.0,11230
11230,Beautiful experience and amazing architecture,This is a great place to visit even quickly if...,2015-01-01,59,5.0,11231


## Create Dimensions


### 1. Time Dimension

In [50]:
# Time dimension
# TimeID
# StayDate
# StayDateYear
# StayDateMonth
# StayDateDay
# StayDateDayOfWeek
# StayDateWeek

db_tripadvisor = mysql.connector.connect(
	host=db_host,
	user=db_user,
	passwd=db_pw,
	database=db_name
)

time_sql = f'''
SELECT OverallID,
            tripadvisor_reviews.DateOfStay AS StayDate,
            YEAR(DateOfStay) AS StayDateYear, 
            MONTH(DateOfStay) AS StayDateMonth, 
            Day(DateOfStay) AS StayDateDay, 
            IF((DayOfWeek(DateOfStay) - 1) = 0, 7, DayOfWeek(DateOfStay) - 1) As StayDateDayOfWeek, 
            WEEK(DateOfStay) AS StayDateWeek
FROM tripadvisor_reviews
'''

df = pd.read_sql(sql=time_sql, con=db_tripadvisor)
df['TimeID'] = df['StayDate'].apply(lambda x: str(uuid4())[:12])
cols = df.columns.to_list()
cols = cols[-1:] + cols[:-1]
df = df[cols]
# change to datetime type
df["StayDate"] = pd.to_datetime(df['StayDate'], format='%Y-%m-%d')
df

,TimeID,OverallID,StayDate,StayDateYear,StayDateMonth,StayDateDay,StayDateDayOfWeek,StayDateWeek
0,b3c1e1a4-095,1,2024-03-01,2024,3,1,5,8
1,b80d9e00-66d,2,2024-03-01,2024,3,1,5,8
2,2b208365-3e8,3,2024-03-01,2024,3,1,5,8
3,09be1ff7-9a1,4,2024-03-01,2024,3,1,5,8
4,eed43f03-9d4,5,2024-03-01,2024,3,1,5,8
...,...,...,...,...,...,...,...,...
11227,9d572247-72b,11228,2014-08-01,2014,8,1,5,30
11228,9f12a0e0-4bf,11229,2015-03-01,2015,3,1,7,9
11229,d28ec9c5-9a3,11230,2015-01-01,2015,1,1,4,0
11230,5791d137-9bc,11231,2015-01-01,2015,1,1,4,0


In [51]:
# Load Time Dimension 
df.to_sql(name='time', con = dwh, if_exists='replace')
dwh.commit()

### 2. Review Dimension

In [52]:
# to clean reviews
def process_text(text):
    
    # Initialise
    lemmatizer = WordNetLemmatizer()
    processed_text = " "
    
    # Process input
    text_lower = text.lower()
    word = word_tokenize(text_lower)
    
    # Alphabetical Tokens
    alphabetic_tokens = [word for word in word if re.match('^[a-zA-Z]+$', word)]
    
    # Remove stopwords from text and lemmatize
    stop_words = set(stopwords.words('english'))

    
    
    lem_words = []
    for word in alphabetic_tokens:
        if word not in stop_words:
            lem_words.append(lemmatizer.lemmatize(word))
    
    # Join the list of words
    processed_text = processed_text.join(lem_words)     #print(edited_stop_words)

    return processed_text

def clean_text(text):
    cleaned_text = ''.join([char.lower() for char in text if char.isalpha() or char.isspace()])
    return cleaned_text


In [53]:
def scale_reviews(value):
    # Calculate the scaled value using linear scaling
    scaled_value = ((value - 1) / (5 - 1)) * (1 - (-1)) + (-1)
    return scaled_value

def weighed_title_score(row):
    analyzer = SentimentIntensityAnalyzer()
    vader = analyzer.polarity_scores(row['CleanReviewTitle'])['compound']
    textblob = TextBlob(row['CleanReviewTitle']).sentiment.polarity
    rating = scale_reviews(row["Rating"])
    avg = (vader + textblob + rating) / 3
    return avg

def weighed_text_score(row):
    analyzer = SentimentIntensityAnalyzer()
    vader = analyzer.polarity_scores(row['CleanReviewText'])['compound']
    textblob = TextBlob(row['CleanReviewText']).sentiment.polarity
    rating = scale_reviews(row["Rating"])
    avg = (vader + textblob + rating) / 3
    return avg

def threshold(value):
    if value < 0:
        return 0
    else:
        return 1

In [54]:
# Review Dimension
# ReviewID
# ReviewText
# ReviewTitle
# ReviewRating

db_tripadvisor = mysql.connector.connect(
	host=db_host,
	user=db_user,
	passwd=db_pw,
	database=db_name
)

review_sql = f"""
    SELECT ReviewTitle, ReviewText, AuthorContribution, Rating, OverallID
    FROM tripadvisor_reviews
"""
df
df = pd.read_sql(sql=review_sql, con=db_tripadvisor)
df['ReviewID'] = df['ReviewTitle'].apply(lambda x: str(uuid4())[:12])
cols = df.columns.to_list()
cols = cols[-1:] + cols[:-1]
df = df[cols]
df['Rating'] = df['Rating'].astype(int)
df['CleanReviewTitle'] = df['ReviewTitle'].apply(process_text)
df['CleanReviewText'] = df['ReviewText'].apply(process_text)
df['WeightedTitleScore'] = df.apply(weighed_title_score, axis = 1)
df['WeightedTextScore'] = df.apply(weighed_text_score, axis = 1)
df.drop(['ReviewTitle', 'ReviewText'], axis=1, inplace=True)



In [55]:
# Load review Dimension 
df.to_sql(name='review', con = dwh, if_exists='replace')
dwh.commit()

In [56]:
# Ingest fact
fact_sql = f'''
    SELECT review.OverallID, review.ReviewID, time.TimeID, review.WeightedTextScore, review.WeightedTitleScore
    FROM review
    INNER JOIN time ON review.OverallID = time.OverallID
'''
df = pd.read_sql(sql = fact_sql, con=dwh)

df["Text_Sentiment"] = df['WeightedTextScore'].apply(threshold)
df["Title_Sentiment"] = df['WeightedTitleScore'].apply(threshold)

df.drop(['WeightedTextScore', 'WeightedTitleScore'], axis=1, inplace=True)

In [57]:
df.to_sql(name='fact', con=dwh, if_exists='replace')
dwh.commit()

In [58]:
db_tripadvisor.close()
dwh.close()

In [59]:
#  housekeeping - set up primary and foreign keys in datawarehouse

db_datawarehouse = mysql.connector.connect(
	host=dwh_host,
	user=dwh_user,
	passwd=dwh_pw,
	database=dwh_name,
    auth_plugin=dwh_pw
)

cursor = db_datawarehouse.cursor()

cursor.execute('ALTER TABLE review ADD PRIMARY KEY (OverallID);')
cursor.execute('ALTER TABLE time ADD PRIMARY KEY (OverallID);')
cursor.execute('ALTER TABLE fact ADD PRIMARY KEY (OverallID);')
cursor.execute('ALTER TABLE fact ADD FOREIGN KEY (OverallID) REFERENCES review(OverallID);')
cursor.execute('ALTER TABLE fact ADD FOREIGN KEY (OverallID) REFERENCES time(OverallID);')


db_datawarehouse.commit()
db_datawarehouse.close()

In [60]:
# check
db_datawarehouse = mysql.connector.connect(
	host=dwh_host,
	user=dwh_user,
	passwd=dwh_pw,
	database=dwh_name,
    auth_plugin=dwh_pw
)

time_Sql = '''
SELECT * FROM time ORDER BY OverallID
'''
df = pd.read_sql(sql=time_Sql, con=db_datawarehouse)

df

,index,TimeID,OverallID,StayDate,StayDateYear,StayDateMonth,StayDateDay,StayDateDayOfWeek,StayDateWeek
0,0,b3c1e1a4-095,1,2024-03-01,2024,3,1,5,8
1,1,b80d9e00-66d,2,2024-03-01,2024,3,1,5,8
2,2,2b208365-3e8,3,2024-03-01,2024,3,1,5,8
3,3,09be1ff7-9a1,4,2024-03-01,2024,3,1,5,8
4,4,eed43f03-9d4,5,2024-03-01,2024,3,1,5,8
...,...,...,...,...,...,...,...,...,...
11227,11227,9d572247-72b,11228,2014-08-01,2014,8,1,5,30
11228,11228,9f12a0e0-4bf,11229,2015-03-01,2015,3,1,7,9
11229,11229,d28ec9c5-9a3,11230,2015-01-01,2015,1,1,4,0
11230,11230,5791d137-9bc,11231,2015-01-01,2015,1,1,4,0


In [61]:
fact_Sql = '''
SELECT * FROM fact ORDER BY OverallID
'''
df = pd.read_sql(sql=fact_Sql, con=db_datawarehouse)

df

,index,OverallID,ReviewID,TimeID,Text_Sentiment,Title_Sentiment
0,0,1,06480f00-da2,b3c1e1a4-095,1,1
1,1,2,eedb2545-2a0,b80d9e00-66d,1,1
2,2,3,884a2ae8-c6e,2b208365-3e8,1,1
3,3,4,ab30aaf7-c7b,09be1ff7-9a1,1,1
4,4,5,37386b04-e39,eed43f03-9d4,1,1
...,...,...,...,...,...,...
11227,9384,11228,4ad46ed9-16a,9d572247-72b,1,1
11228,11231,11229,6c630012-c67,9f12a0e0-4bf,1,1
11229,5784,11230,da15b9cf-2c9,d28ec9c5-9a3,1,1
11230,9385,11231,dee1102a-0af,5791d137-9bc,1,1


In [62]:
review_Sql = '''
SELECT * FROM review ORDER BY OverallID
'''
df = pd.read_sql(sql=review_Sql, con=db_datawarehouse)

df

,index,ReviewID,AuthorContribution,Rating,OverallID,CleanReviewTitle,CleanReviewText,WeightedTitleScore,WeightedTextScore
0,0,06480f00-da2,73,5,1,must see singapore,must miss place tourist visit singapore pretty...,0.333333,0.719524
1,1,eedb2545-2a0,3,5,2,recently rejuvenating spa,recently rejuvenating spa experience hour flig...,0.333333,0.759911
2,2,884a2ae8-c6e,2,5,3,professional service,visited hotel drink really wanted say wonderfu...,0.366667,0.778333
3,3,ab30aaf7-c7b,1,5,4,marina bay world class,amazing hotel loved facility someone enjoys gy...,0.333333,0.872200
4,4,37386b04-e39,2,5,5,nice touch,initial check experience expected nicole manag...,0.673833,0.715333
...,...,...,...,...,...,...,...,...,...
11227,11227,4ad46ed9-16a,97,5,11228,good hotel,nice location near shopping mall big swimming ...,0.713467,0.594843
11228,11228,6c630012-c67,13,4,11229,swim heaven,everytime check sand love going pool enjoy lon...,0.336867,0.587251
11229,11229,da15b9cf-2c9,68,5,11230,unmatcheable property,stayed night hotel real superb beauty jewel si...,0.333333,0.777078
11230,11230,dee1102a-0af,59,5,11231,beautiful experience amazing architecture,great place visit even quickly hour layover ai...,0.850700,0.801139
